<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Eye_ai_get_cropped_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Cropping

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the cropped images' bounding box to the catalog.

In [1]:
# Prerequisites to configure colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install deriva
    !pip install bdbag
    !pip install --upgrade --force pydantic
    !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
    !pip install setuptools_git_versioning


In [2]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [4]:
import json
import os
import eye_ai
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
print(f"Using eye_ai version: {eye_ai.__version__}")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

Using eye_ai version: 1.1.0.post11+git.a4520290


In [6]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = True #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}

You are already logged in.


In [18]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-9F8W"      # Configuration file for this run.  Needs to be changed for each execution.

In [8]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)


2024-05-03 11:35:01,198 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-03 11:35:01,199 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [19]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()

2024-05-03 11:45:00,186 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-test_cropping.json] transfer successful. 0.69 KB transferred. Elapsed time: 0:00:00.000052.
2024-05-03 11:45:00,187 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-test_cropping.json]
2024-05-03 11:45:00,201 - INFO - Configuration validation successful!
2024-05-03 11:45:06,856 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Assets/optic_disk_crop_model.hdf5] transfer successful. 76.75 MB transferred at 15.72 MB/second. Elapsed time: 0:00:04.882836.
2024-05-03 11:45:06,857 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Assets/optic_disk_crop_model.hdf5]


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/lizhiwei/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'Image Cropping', 'rid': '2-5NRG'}],
  'Annotation_Type': [{'name': 'Optic Nerve', 'rid': '2-5PWP'}]},
 'execution_rid': '2-9F8Y',
 'workflow_rid': '2-9F90',
 'bag_paths': [PosixPath('/data/2-5K94_a0c5a1bf0b299df693e906b0314883cdb52b6c29f7a3493fde66e3e5d10dca00/Dataset_2-5K94')],
 'assets_paths': [PosixPath('/data/lizhiwei/EyeAI_working/Execution_Assets/optic_disk_crop_model.hdf5')],
 'configuration_path': PosixPath('/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-test_cropping.json')}

In [20]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
field2_csv_path = EA.filter_angle_2(configuration_records.bag_paths[0])
field2_csv_path

PurePosixPath('/data/lizhiwei/EyeAI_working/Field_2.csv')

In [22]:
EA.execution_assets_path

PosixPath('/data/lizhiwei/EyeAI_working/Execution_Assets')

In [23]:
# @title Execute Proecss algorithm (Cropping)
from eye_ai_ml.glaucoma.optic_disk_crop import preprocess_and_crop
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  preprocess_and_crop(
      str(configuration_records.bag_paths[0])+"/data/assets/Image/",
      field2_csv_path,
      str(EA.working_dir)+'/output.csv',
      'template.jpg',
      str(EA.execution_assets_path),
      configuration_records.assets_paths[0],
      configuration_records.vocabs['Annotation_Type'][0].rid,
      configuration_records.vocabs['Annotation_Type'][0].name,
      False
      )

2024-05-03 11:52:51.295216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 11:52:51.295267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 11:52:51.296079: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 11:52:51.302389: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 11:52:52.107586: W tensorflow/compiler/tf2

2024-05-03 11:52:52.802239: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 11:52:52.831672: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 11:52:52.833306: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

80134624/80134624 [==============================] - 1s 0us/step


2024-05-03 11:52:54,538 - INFO - Processing image 432880_701304_8649867_Left Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary
2024-05-03 11:52:55.480244: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 1s 747ms/step


...

2024-05-03 11:53:07,306 - INFO - SVG for AE4R saved.
2024-05-03 11:53:07,411 - INFO - Processing image 974325_1401774_14092247_Left Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary


1/1 [==============================] - 0s 21ms/step


2024-05-03 11:53:08,098 - INFO - SVG for AFQY saved.
2024-05-03 11:53:08,205 - INFO - Processing image 974325_1401774_14092253_Right Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary


1/1 [==============================] - 0s 20ms/step


2024-05-03 11:53:08,918 - INFO - SVG for AFR6 saved.
2024-05-03 11:53:08,921 - INFO - Number of images in CSV: 14
2024-05-03 11:53:08,921 - INFO - Number of images in directory: 42
2024-05-03 11:53:08,922 - INFO - Number of images in output directory: 2
2024-05-03 11:53:08,923 - INFO - Number of cropped images: 14
2024-05-03 11:53:08,923 - INFO - --- 14.491947174072266 seconds ---


In [24]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)


2024-05-03 11:53:27,591 - INFO - Initializing uploader: GenericUploader v1.7.0 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-05-03 11:53:27,592 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-03 11:53:27,593 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-05-03 11:53:27,608 - INFO - Checking for updated configuration...
2024-05-03 11:53:27,721 - INFO - Updated configuration found.
2024-05-03 11:53:27,722 - INFO - Scanning files in directory [/data/lizhiwei/EyeAI_working/Execution_Assets/Execution_Assets]...
2024-05-03 11:53:27,726 - INFO - Including file: [/data/lizhiwei/EyeAI_working/Execution_Assets/Execution_Assets/Image_Annotation/Cropped_7RBG.svg].
2024-05-03 11:53:27,727 - INFO - Including file: [/data/lizhiwei/EyeAI_working/Execution_A